# STAR Mapping

We load the genome into memory first to prevent needing this once per sample, and do it once in general.

STAR will use the whitelist generated previously per sample to identify HDMIs and assign transcripts to them

In [ ]:
cd /path/to/spatial/data

In [ ]:
GENOME_DIR=indexes/STAR/2.7.10a/mm10

In [ ]:
STAR --genomeLoad Remove --genomeDir ${GENOME_DIR}
STAR --genomeLoad LoadAndExit --genomeDir ${GENOME_DIR}
rm -r _STARtmp Log.out Log.progress.out Aligned.out.sam

In [ ]:
for sample in `ls Demultiplexed/*_R1_001.fastq.gz | grep -v Undeter | cut -d'/' -f2 | sed 's/_S[0-9]_R1_001.fastq.gz//'`; do
    BARCODE_WHITELIST_FILE=output/whitelist_${sample}*
    mkdir -p ${sample}
    STAR \
        --soloType CB_UMI_Simple \
        --soloCBwhitelist ${BARCODE_WHITELIST_FILE} \
        --soloCBstart 1 \
        --soloCBlen 31 \
        --soloUMIstart 32 \
        --soloUMIlen 9 \
        --soloBarcodeMate 0 \
        --soloBarcodeReadLength 0 \
        --soloFeatures Gene GeneFull \
        --soloCBmatchWLtype 1MM \
        --soloUMIdedup 1MM_All \
        --soloCellFilter None \
        --outSAMtype BAM SortedByCoordinate \
        --outSAMattributes NH HI AS nM CR CY UR UY CB UB sS \
        --runThreadN 36 \
        --genomeDir ${GENOME_DIR} \
        --genomeLoad LoadAndKeep \
        --limitBAMsortRAM 50000000000 \
        --readFilesIn Demultiplexed_32_80/${sample}_S1_R2_001.fastq.gz Demultiplexed/${sample}_S1_R1_UMI_added.fastq.gz \
        --readFilesCommand zcat \
        --outFileNamePrefix output/${sample}/${sample}_ \
        --outReadsUnmapped Fastx
done

# readFilesIn should be two lists of comma seperated values. The first should be Read 2 files and the second should be read 1 files
# STARsolo can only do error corretion with 31 bases, so we ignore the last base.

In [ ]:
STAR --genomeLoad Remove --genomeDir ${GENOME_DIR}